In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

/Users/bh0118/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/bh0118/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='

In [2]:
data = pd.read_excel('./birthweight_low.xlsx')

In [3]:
data.head(10)

,mage,meduc,monpre,npvis,fage,feduc,omaps,fmaps,cigs,drink,male,mwhte,mblck,moth,fwhte,fblck,foth,bwght
0,69,NaN,5,2.0,62,NaN,4,7,23,9,1,0,1,0,0,1,0,697
1,68,12.0,3,10.0,61,11.0,4,6,25,11,1,1,0,0,1,0,0,1290
2,71,12.0,3,6.0,46,12.0,2,7,21,12,1,0,1,0,0,1,0,1490
3,59,16.0,1,8.0,48,16.0,7,8,21,10,0,0,0,1,0,0,1,1720
4,48,12.0,4,6.0,39,12.0,2,9,17,13,0,1,0,0,1,0,0,1956
5,67,11.0,4,8.0,40,8.0,4,9,16,14,0,1,0,0,1,0,0,1984
6,54,12.0,2,12.0,46,12.0,9,9,17,12,1,0,1,0,0,1,0,2050
7,71,14.0,4,7.0,51,11.0,9,8,15,13,0,1,0,0,1,0,0,2068
8,56,12.0,1,9.0,53,14.0,8,9,14,9,1,1,0,0,1,0,0,2148
9,58,12.0,2,12.0,61,16.0,9,9,13,6,0,0,1,0,0,1,0,2180


## 1. Data preprocessing
#### Fill missing value

In [4]:
data.isna().sum()

mage      0
meduc     3
monpre    0
npvis     3
fage      0
feduc     7
omaps     0
fmaps     0
cigs      0
drink     0
male      0
mwhte     0
mblck     0
moth      0
fwhte     0
fblck     0
foth      0
bwght     0
dtype: int64

#### Since all the columns with missing values are continuous, we fill in them with mean value

In [5]:
data[['meduc','npvis','feduc']] = data[['meduc','npvis','feduc']].fillna(value = data[['meduc','npvis','feduc']].mean())

#### To avoid dummy variable trap, in [mwhte, mblck, moth] we drop one of them, say mblck to avoid multicolinearity. We do the same to [fwhte, fblck, foth]

In [6]:
data = data.drop(['mblck','fblck'],axis=1)

## Use VIF to justify multicolinarity

In [7]:
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [8]:
calc_vif(data.iloc[:,:-1])

,variables,VIF
0,mage,27.882915
1,meduc,82.187370
2,monpre,5.463062
3,npvis,9.960589
4,fage,30.814372
5,feduc,64.201143
6,omaps,51.426889
7,fmaps,142.028576
8,cigs,5.287045
9,drink,5.676175


#### For high VIFs, drop meduc, feduc since they are not quite correlated with birthweight. Drop omaps since fmaps contains more info.
#### Drop fage and fmaps since they aree highly correlated with mage. Drop 'moth','foth','fwhte' since they aree higher than 5

In [9]:
data = data.drop(['mage','meduc','feduc','fage','fmaps','omaps','moth','foth','fwhte'],axis=1)
#data['increase_maps']=data.apply(lambda x:x['fmaps']-x['omaps'],axis=1)
#data = data.drop([],axis=1)

In [10]:
calc_vif(data.drop('bwght',axis=1))

,variables,VIF
0,monpre,2.909303
1,npvis,4.544878
2,cigs,4.559596
3,drink,4.588671
4,male,2.045500
5,mwhte,1.436291


In [11]:
data.corr()

,monpre,npvis,cigs,drink,male,mwhte,bwght
monpre,1.000000,-0.342175,-0.058436,0.015553,-0.075269,0.075486,-0.051827
npvis,-0.342175,1.000000,0.036211,-0.067171,0.002164,0.041190,0.061358
cigs,-0.058436,0.036211,1.000000,0.355779,-0.115098,0.033570,-0.572385
drink,0.015553,-0.067171,0.355779,1.000000,-0.092433,0.164619,-0.743125
male,-0.075269,0.002164,-0.115098,-0.092433,1.000000,-0.097089,0.114855
mwhte,0.075486,0.041190,0.033570,0.164619,-0.097089,1.000000,-0.114388
bwght,-0.051827,0.061358,-0.572385,-0.743125,0.114855,-0.114388,1.000000


In [12]:
data

,monpre,npvis,cigs,drink,male,mwhte,bwght
0,5,2.0,23,9,1,0,697
1,3,10.0,25,11,1,1,1290
2,3,6.0,21,12,1,0,1490
3,1,8.0,21,10,0,0,1720
4,4,6.0,17,13,0,1,1956
...,...,...,...,...,...,...,...
191,5,12.0,3,2,1,0,4610
192,2,12.0,10,4,1,0,4660
193,2,15.0,0,1,0,1,4678
194,2,6.0,8,4,1,0,4791


#### Now all the VIFs are at the normal range

In [13]:
y = data['bwght']
X = data[['monpre', 'npvis', 'cigs', 'drink', 'male', 'mwhte'
       ]]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size=0.25, random_state=219)

## 2. Model fit
#### Linear regression

In [15]:
lm = LinearRegression()
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
lm.score(X_test,y_test)

0.7063343591559027

In [17]:
np.abs(lm.score(X_train,y_train)- lm.score(X_test,y_test))

0.05557412353419733

In [18]:
lm.score(X_test,y_test) - (np.abs(lm.score(X_train,y_train)- lm.score(X_test,y_test))-0.05)

0.7007602356217054

#### Lasso regression

In [19]:
alpha_list = [0.05*i for i in range(60)]
lasso_train_r2 = []
lasso_test_r2 = []
lasso_train_test_gap = []
lasso_final_score = []
# Tune parameteer with alpha = (0,3)
for alp in alpha_list:
    clf = linear_model.Lasso(alpha=alp)
    clf.fit(X_train,y_train)
    lasso_train_r2.append(clf.score(X_train,y_train))
    lasso_test_r2.append(clf.score(X_test,y_test))
    lasso_train_test_gap.append(np.abs(clf.score(X_train,y_train)- clf.score(X_test,y_test)))
    lasso_final_score.append(clf.score(X_test,y_test)-(np.abs(clf.score(X_train,y_train)- clf.score(X_test,y_test))-0.05))

/Users/bh0118/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  if __name__ == '__main__':
/Users/bh0118/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/bh0118/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11445279.594996076, tolerance: 6554.396585034014
  positive)


In [20]:
li = {'alpha':alpha_list, 'lasso_test_r2':lasso_test_r2, 'lasso_train_test_gap':lasso_train_test_gap,'lasso_final_score':lasso_final_score}
lasso_res = pd.DataFrame(li)

In [21]:
#final score has the substraction of part over 0.05
lasso_res

,alpha,lasso_test_r2,lasso_train_test_gap,lasso_final_score
0,0.00,0.706334,0.055574,0.700760
1,0.05,0.706435,0.055675,0.700760
2,0.10,0.706536,0.055776,0.700760
3,0.15,0.706636,0.055876,0.700760
4,0.20,0.706736,0.055977,0.700759
5,0.25,0.706837,0.056078,0.700759
6,0.30,0.706937,0.056178,0.700758
7,0.35,0.707036,0.056279,0.700757
8,0.40,0.707136,0.056380,0.700756
9,0.45,0.707235,0.056480,0.700755


#### ARD

In [22]:
ARD_clf = linear_model.ARDRegression(tol = 1e-10,alpha_1=1e-10,alpha_2=1e-10,lambda_1=1e-10,lambda_2=1e-10)

In [23]:
ARD_clf.fit(X_train,y_train)
ARD_clf.score(X_test,y_test)

0.7284494369869126

In [24]:
np.abs(ARD_clf.score(X_train,y_train)- ARD_clf.score(X_test,y_test))

0.08208054575637069

In [25]:
ARD_clf.score(X_test,y_test) - (np.abs(ARD_clf.score(X_train,y_train)- ARD_clf.score(X_test,y_test))-0.05)

0.6963688912305419

#### KNN Regression

In [26]:
neigh_list=[i for i in range(5,61)]
KNN_test_r2 = []
KNN_train_test_gap = []
for ne in neigh_list:
    neigh = KNeighborsRegressor(n_neighbors=ne)
    neigh.fit(X_train,y_train)
    KNN_test_r2.append(neigh.score(X_test,y_test))
    KNN_train_test_gap.append(np.abs(neigh.score(X_train,y_train)- neigh.score(X_test,y_test)))

In [27]:
ki = {'n_neighbour':neigh_list, 'knn_test_r2':KNN_test_r2, 'knn_train_test_gap':KNN_train_test_gap}
knn_res = pd.DataFrame(ki)

In [28]:
# Result with n_neighbour = 5-60
knn_res

,n_neighbour,knn_test_r2,knn_train_test_gap
0,5,0.496364,0.188765
1,6,0.527917,0.158190
2,7,0.528291,0.156743
3,8,0.534484,0.145884
4,9,0.542254,0.116844
5,10,0.531270,0.113764
6,11,0.526084,0.111125
7,12,0.554024,0.077850
8,13,0.548515,0.078493
9,14,0.550180,0.065949


In [29]:
#KNN regression performs bad on test cases

## 3. Final Model used: Lasso(alpha=0.05)

In [30]:
final_mod = linear_model.Lasso(alpha=0.05)
final_mod.fit(X_train,y_train)
final_mod.score(X_train,y_train)
final_mod.score(X_test,y_test)
np.abs(final_mod.score(X_train,y_train)- final_mod.score(X_test,y_test))
final_mod.score(X_test,y_test)-(np.abs(final_mod.score(X_train,y_train)- final_mod.score(X_test,y_test))-0.05)

0.7007601741364912

In [31]:
ans={}
ans['model type'] = 'Lasso with alpha=0.05, with X = '
ans['train score'] = final_mod.score(X_train,y_train)
ans['test score'] = final_mod.score(X_test,y_test)
ans['train_test_gap']=np.abs(final_mod.score(X_train,y_train)- final_mod.score(X_test,y_test))
ans

{'model type': 'Lasso with alpha=0.05',
 'train score': 0.6507601741364911,
 'test score': 0.7064351156831647,
 'train_test_gap': 0.05567494154667363}